In [3]:
import requests
import pandas as pd
from pathlib import Path
import time
from IPython.display import clear_output

In [22]:
df = pd.read_csv('NA_result.csv')
matchId = df.loc[len(df)-1]['matchId']
matchId = int(matchId[4:])
print('Last matchId in csv:',matchId)
matchId = matchId-1
print('First matchId in new scan:', matchId)

Last matchId in csv: 4619508349
First matchId in new scan: 4619508348


In [23]:
key = 'RGAPI-331d979e-3c9d-41cb-bc17-07f1d78ec2ef'
url = 'https://americas.api.riotgames.com/lol/match/v5/matches/'
query = '{}NA1_{}?api_key={}'.format(url, matchId+1, key)
# query = '{}NA1_{}?api_key={}'.format(url, 4619747330, key)

response = requests.get(query)
json = response.json()
display(json['info']['teams'][0]['win'])

False

In [ ]:
countAdded = 0
countTried = 1
startTime = round(time.time())

while(countAdded < 50000):
  query = '{}NA1_{}?api_key={}'.format(url, matchId, key)
  response = requests.get(query)
  json = response.json()

  if response.status_code == 200:
    if json['info']['gameMode'] != 'ARAM':

      participants = json['info']['participants'][::]
      # gameId = json['metadata']['matchId']

      champIds = []
      champNames = []
      index = 1
      for p in participants:
        
        champIds.append(p['championId'])
        champNames.append(p['championName'])

      champs = {'matchId': json['metadata']['matchId'],
                'blue1':champIds[0],
                'blue2':champIds[1],
                'blue3':champIds[2],
                'blue4':champIds[3],
                'blue5':champIds[4],
                'red1':champIds[5],
                'red2':champIds[6],
                'red3':champIds[7],
                'red4':champIds[8],
                'red5':champIds[9],
                'blueWin':json['info']['teams'][0]['win'] 
      }
      
      
      df.loc[len(df)] = champs
      print('matchId {} added, {} total matches added.'.format(json['metadata']['matchId'], countAdded))
      countAdded += 1
  matchId -= 1
  countTried += 1
  if countTried == 99:
    timeElasped = round(time.time()) - startTime
    timeToWait = 120 - timeElasped
    if timeToWait < 0:
        timeToWait = 0
    print('Waiting for', timeToWait, 'seconds to start again')
    df.to_csv('NA_result.csv')
    time.sleep(timeToWait+2)
    print('Restarting queries')
    startTime = round(time.time())
    countTried = 1
    clear_output(wait=True)
# df_champions.drop([0])
# display(df_champions)

Waiting for 94 seconds to start again
